In [23]:
import yaml
import itertools

with open('sweep_definition.yaml', 'r') as file:
    sweep_config = yaml.safe_load(file)

dictionary = sweep_config['search_space']
choices = [{key:sweep_config['search_space'][key].get("values")} for key in dictionary.keys() if sweep_config['search_space'][key]]
keys = [list(d.keys())[0] for d in choices]
values = [list(d.values())[0] for d in choices]

combinations = [dict(zip(keys, combination)) for combination in itertools.product(*values)]

In [24]:
combinations

[{'aoai_deployment_name': 'gpt4-vh'},
 {'aoai_deployment_name': 'gpt35-turbo-1106'}]

In [26]:
import json
import subprocess

file_path = "./rag_flow_template/cookiecutter.json"
with open(file_path, "r") as file:
    data = json.load(file)

for i, search_step_dict in enumerate(combinations):
    data["flow_name"] = f"grid_step_{i}"
    for key in ["aoai_deployment_name", "ai_search_index_name"]:
        if key in search_step_dict.keys():
            data[key] = search_step_dict[key]
    print(data)
    with open(file_path, "w") as file:
        json.dump(data, file, indent=4)
    
    subprocess.run(["cd ./flow_versions && cookiecutter ../rag_flow_template --no-input --skip-if-file-exists"], shell=True)


{'aoai_deployment_name': 'gpt4-vh', 'flow_name': 'grid_step_0', '_copy_without_render': ['*.jinja2', '.promptflow/*']}
{'aoai_deployment_name': 'gpt35-turbo-1106', 'flow_name': 'grid_step_1', '_copy_without_render': ['*.jinja2', '.promptflow/*']}


In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from promptflow.azure import PFClient


credential = DefaultAzureCredential()
  
# Get a handle to workspace, it will use config.json in current and parent directory.
pf = PFClient.from_config(
    credential=credential,
)

Found the config file in: /home/vhoudebine/config.json


In [27]:
import os

def create_and_run_flow(i):
    # Apply your function to the dictionary here
    flow = os.path.join("./flow_versions", f"rag_flow_grid_step_{i}")
    data ="./data/data.jsonl"
    print(f"Creating run for {flow}")
    # create run
    base_run = pf.run(
        flow=flow,
        data=data,
        column_mapping={
        "question": "${data.question}"
        }
    )
    print(f"Created run {base_run.name}")
    return base_run


# Create a thread for each dictionary
runs = []
for i, search_step_dict in enumerate(combinations):
    run = create_and_run_flow(i)
    runs.append(run)

Creating run for ./flow_versions/rag_flow_grid_step_0


Uploading rag_flow_grid_step_0 (0.0 MBs): 100%|██████████| 514/514 [00:00<00:00, 4185.59it/s]


[2024-04-04 20:04:31,062][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ml.azure.com/runs/rag_flow_grid_step_0_variant_0_20240404_200429_297972?wsid=/subscriptions/6c065ea7-65cd-4a34-8e2a-3e21ad4a8e9f/resourcegroups/vince-rg/providers/Microsoft.MachineLearningServices/workspaces/vince-dev
Created run rag_flow_grid_step_0_variant_0_20240404_200429_297972
Creating run for ./flow_versions/rag_flow_grid_step_1


[2024-04-04 20:04:42,410][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ml.azure.com/runs/rag_flow_grid_step_1_variant_0_20240404_200441_308304?wsid=/subscriptions/6c065ea7-65cd-4a34-8e2a-3e21ad4a8e9f/resourcegroups/vince-rg/providers/Microsoft.MachineLearningServices/workspaces/vince-dev
Created run rag_flow_grid_step_1_variant_0_20240404_200441_308304


In [28]:
run = runs[0]